# Elastic Constants

Solids respond to small external loads through a reversible elastic response. The strength of the response is characterized by the elastic moduli. {py:mod}`matscipy.elasticity` implements functions for computing elastic moduli from small deformation of atomistic systems that consider potential symmetries of the underlying atomic system, in particular for crystals. {py:mod}`matscipy.elasticity` also implements analytic calculation of elastic moduli for some interatomic potentials, described in more detail below. The computation of elastic moduli is a basic prerequisite for multi-scale modelling of materials, as they are the most basic parameters of continuum material models.

In this tutorial, we will go over different ways that `matscipy` can compute elastic constants of an atomic configuration.

## Problem setup